<a href="https://notebooks.gesis.org/binder/v2/gh/monarch-beluga/Geemap_Tutorials/master?filepath=Image%2F03_Image%E5%8F%AF%E8%A7%86%E5%8C%96%E5%8F%82%E6%95%B0%E4%BA%86%E8%A7%A3.ipynb" target="_parent">
<img src="https://notebooks.gesis.org/binder/badge_logo.svg" alt="launch binder"/>
</a>

In [ ]:
# !pip install geemap

In [ ]:
import geemap
import ee

In [ ]:
Map = geemap.Map()
Map

In [ ]:
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318').select('B3', 'B4', 'B5')
vizParams = {
  'bands': ['B5', 'B4', 'B3'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}
Map.setCenter(-122.1899, 37.5010, 8)      # 按地图视图按经纬度坐标来
Map.addLayer(image, vizParams, 'false color composite')
# bands参数需要一个有且只含三个元素的列表，列表里每一个参数为影像的一个波段的名称
# 用这三个波段来映射到RGB的红绿蓝三种颜色中
# min,max设置显示波段值的颜色范围
# gamma，即用于gamma矫正的一些参数，以矫正图像

In [ ]:
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')
ndwi = image.normalizedDifference(['B3', 'B5'])
# 计算两个波段之间归一化差异计算方法为两个波段之差除以两个波段之和
ndwiViz = {'min': 0.5, 'max': 1, 'palette': ['00FFFF', '0000FF']}
Map.setCenter(-122.1899, 37.5010, 8)
Map.addLayer(ndwi, ndwiViz, 'NDWI')
# palette，用于以彩色显示单波段的图像，由CSS样式的颜色字符串列表来建立色带显示图像

In [ ]:
ndwiMasked = ndwi.updateMask(ndwi.gte(0.4))
Map.addLayer(ndwiMasked, ndwiViz, 'NDWI masked')
# updateMask,将不符合要求的点在图像上设置成不透明度为0

In [ ]:
imageRGB = image.visualize(**{'bands': ['B5',  'B4',  'B3'], 'max': 0.5})
ndwiRGB = ndwiMasked.visualize(**{
  'min': 0.5,
  'max': 1,
  'palette': ['00FFFF', '0000FF']
})
Map.addLayer(imageRGB, {}, "imageRGB")
Map.addLayer(ndwiRGB, {}, "ndwiRGB")
# visualize可视化图像的RGB和灰度值

In [ ]:
mosaic = ee.ImageCollection([imageRGB, ndwiRGB]).mosaic()
Map.addLayer(mosaic, {}, 'mosaic')
# 先用ImageCollection建立影像集，然后用.mosaic()按集合中的影像顺序绘制图层
# 实现一种镶嵌的效果

In [ ]:
roi = ee.Geometry.Point([-122.4481, 37.7599]).buffer(20000)
Map.addLayer(mosaic.clip(roi))
# clip为裁剪图层

In [ ]:
cover = ee.Image('MODIS/051/MCD12Q1/2012_01_01') \
  .select('Land_Cover_Type_1')

igbpPalette = [
  'aec3d4', # water
  '152106', '225129', '369b47', '30eb5b', '387242', # forest
  '6a2325', 'c3aa69', 'b76031', 'd9903d', '91af40',  # shrub, grass
  '111149', # wetlands
  'cdb33b', # croplands
  'cc0013', # urban
  '33280d', # crop mosaic
  'd7cdcc', # snow and ice
  'f7e084', # barren
  '6f6f6f'  # tundra
]

Map.setCenter(-99.229, 40.413, 5)
Map.addLayer(cover,
             {'min': 0, 'max': 17, 'palette': igbpPalette},
             'IGBP classification')

In [ ]:

cover = ee.Image('MODIS/051/MCD12Q1/2012_01_01').select('Land_Cover_Type_1')

# Define an SLD style of discrete intervals to apply to the image.
sld_intervals = \
'<RasterSymbolizer>' + \
 ' <ColorMap type="intervals" extended="False" >' + \
    '<ColorMapEntry color="#aec3d4" quantity="0" label="Water"/>' + \
    '<ColorMapEntry color="#152106" quantity="1" label="Evergreen Needleleaf Forest"/>' + \
    '<ColorMapEntry color="#225129" quantity="2" label="Evergreen Broadleaf Forest"/>' + \
    '<ColorMapEntry color="#369b47" quantity="3" label="Deciduous Needleleaf Forest"/>' + \
    '<ColorMapEntry color="#30eb5b" quantity="4" label="Deciduous Broadleaf Forest"/>' + \
    '<ColorMapEntry color="#387242" quantity="5" label="Mixed Deciduous Forest"/>' + \
    '<ColorMapEntry color="#6a2325" quantity="6" label="Closed Shrubland"/>' + \
    '<ColorMapEntry color="#c3aa69" quantity="7" label="Open Shrubland"/>' + \
    '<ColorMapEntry color="#b76031" quantity="8" label="Woody Savanna"/>' + \
    '<ColorMapEntry color="#d9903d" quantity="9" label="Savanna"/>' + \
    '<ColorMapEntry color="#91af40" quantity="10" label="Grassland"/>' + \
    '<ColorMapEntry color="#111149" quantity="11" label="Permanent Wetland"/>' + \
    '<ColorMapEntry color="#cdb33b" quantity="12" label="Cropland"/>' + \
    '<ColorMapEntry color="#cc0013" quantity="13" label="Urban"/>' + \
    '<ColorMapEntry color="#33280d" quantity="14" label="Crop, Natural Veg. Mosaic"/>' + \
    '<ColorMapEntry color="#d7cdcc" quantity="15" label="Permanent Snow, Ice"/>' + \
    '<ColorMapEntry color="#f7e084" quantity="16" label="Barren, Desert"/>' + \
    '<ColorMapEntry color="#6f6f6f" quantity="17" label="Tundra"/>' + \
  '</ColorMap>' + \
'</RasterSymbolizer>'
Map.addLayer(cover.sldStyle(sld_intervals), {}, 'IGBP classification styled')


In [ ]:

image = ee.Image('CGIAR/SRTM90_V4')

sld_intervals = \
  '<RasterSymbolizer>' + \
    '<ColorMap type="intervals" extended="False" >' + \
      '<ColorMapEntry color="#0000ff" quantity="0" label="0"/>' + \
      '<ColorMapEntry color="#00ff00" quantity="100" label="1-100" />' + \
      '<ColorMapEntry color="#007f30" quantity="200" label="110-200" />' + \
      '<ColorMapEntry color="#30b855" quantity="300" label="210-300" />' + \
      '<ColorMapEntry color="#ff0000" quantity="400" label="310-400" />' + \
      '<ColorMapEntry color="#ffff00" quantity="1000" label="410-1000" />' + \
    '</ColorMap>' + \
  '</RasterSymbolizer>'

sld_ramp = \
  '<RasterSymbolizer>' + \
    '<ColorMap type="ramp" extended="False" >' + \
      '<ColorMapEntry color="#0000ff" quantity="0" label="0"/>' + \
      '<ColorMapEntry color="#00ff00" quantity="100" label="100" />' + \
      '<ColorMapEntry color="#007f30" quantity="200" label="200" />' + \
      '<ColorMapEntry color="#30b855" quantity="300" label="300" />' + \
      '<ColorMapEntry color="#ff0000" quantity="400" label="400" />' + \
      '<ColorMapEntry color="#ffff00" quantity="500" label="500" />' + \
    '</ColorMap>' + \
  '</RasterSymbolizer>'

Map.setCenter(-76.8054, 42.0289, 8)
Map.addLayer(image.sldStyle(sld_intervals), {}, 'SLD intervals')
Map.addLayer(image.sldStyle(sld_ramp), {}, 'SLD ramp')

In [ ]:
image = ee.Image('LANDSAT/LC08/C01/T1/LC08_044034_20140318')

template_sld = \
  '<RasterSymbolizer>' + \
    '<ContrastEnhancement><_enhance_/></ContrastEnhancement>' + \
    '<ChannelSelection>' + \
      '<RedChannel>' + \
        '<SourceChannelName>B5</SourceChannelName>' + \
      '</RedChannel>' + \
      '<GreenChannel>' + \
        '<SourceChannelName>B4</SourceChannelName>' + \
      '</GreenChannel>' + \
      '<BlueChannel>' + \
        '<SourceChannelName>B3</SourceChannelName>' + \
      '</BlueChannel>' + \
    '</ChannelSelection>' + \
  '</RasterSymbolizer>'

equalize_sld = template_sld.replace('_enhance_', 'Histogram')
normalize_sld = template_sld.replace('_enhance_', 'Normalize')

Map.centerObject(image, 10)
Map.addLayer(image, {'bands': ['B5',  'B4',  'B3'], 'min': 0, 'max': 15000}, 'Linear')
Map.addLayer(image.sldStyle(equalize_sld), {}, 'Equalized')
Map.addLayer(image.sldStyle(normalize_sld), {}, 'Normalized')
